In [1]:
from z3 import *

# Initialize solver
s = Solver()
N = 4
cards = N**2 + N + 1
symbols = cards
symbols_per_card = N + 1

tot = 0


deck = [[Bool(f"deck_{i}_{j}") for j in range(symbols)] for i in range(cards)]


s += [deck[row][0] == True for row in range(symbols_per_card)]
s += [deck[row][0] == False for row in range(symbols_per_card, cards)]
s += [deck[0][1] == True]
s += [deck[i][1] == False for i in range(1, symbols_per_card)]


s += [
    deck[row][1] == True
    for row in range(symbols_per_card, symbols_per_card + N)
]
s += [
    deck[row][2] == False
    for row in range(symbols_per_card + 1, symbols_per_card + N)
]
s += [deck[row][1] == False for row in range(symbols_per_card + N, cards)]

# Constraints
s += [Sum(card) == symbols_per_card for card in deck]
s += [
    Sum(
        [
            If(And(deck[i][k] == deck[j][k], deck[i][k] == True), 1, 0)
            for k in range(symbols)
        ]
    )
    == 1
    for i in range(cards - 1)
    for j in range(i + 1, cards)
]


binary_values = [
    Sum([If(deck[i][j], 2 ** (symbols - j - 1), 0) for j in range(symbols)])
    for i in range(cards)
]
s += [binary_values[i] > binary_values[i + 1] for i in range(cards - 1)]

# the total number of trues in each column should be N+1 but this is already done for the first column
s += [
    Sum([If(deck[i][j], 1, 0) for i in range(cards)]) == symbols_per_card
    for j in range(2, symbols)
]

# Function to serialize the model


def serialize(grid):
    res = ""
    for row in grid:
        for cell in row:
            res += str(Int(cell))
    return res


# Check for satisfiability and print the serialized result
while s.check() == sat:
    tot += 1
    print(tot)
    m = s.model()
    grid = [[m.eval(deck[i][j]) for j in range(symbols)] for i in range(cards)]
    print("\\begin{align*}")
    for row in grid:
        print("&", end="")
        print(
            "".join(["1" if row[i] else "0" for i in range(symbols)]), end=""
        )
        print("\\\\")
    print("\\end{align*}")
    s += Or(
        [
            deck[i][j] != grid[i][j]
            for j in range(symbols)
            for i in range(cards)
        ]
    )

else:
    print(tot)
    print("unsat")

1
\begin{align*}
&1100001\\
&1010010\\
&1001100\\
&0110100\\
&0101010\\
&0011001\\
&0000111\\
\end{align*}
2
\begin{align*}
&1100100\\
&1010010\\
&1001001\\
&0111000\\
&0100011\\
&0010101\\
&0001110\\
\end{align*}
3
\begin{align*}
&1100010\\
&1010001\\
&1001100\\
&0111000\\
&0100101\\
&0010110\\
&0001011\\
\end{align*}
4
\begin{align*}
&1100100\\
&1010001\\
&1001010\\
&0111000\\
&0100011\\
&0010110\\
&0001101\\
\end{align*}
5
\begin{align*}
&1100010\\
&1010100\\
&1001001\\
&0111000\\
&0100101\\
&0010011\\
&0001110\\
\end{align*}
6
\begin{align*}
&1100001\\
&1010100\\
&1001010\\
&0111000\\
&0100110\\
&0010011\\
&0001101\\
\end{align*}
7
\begin{align*}
&1101000\\
&1010100\\
&1000011\\
&0110001\\
&0100110\\
&0011010\\
&0001101\\
\end{align*}
8
\begin{align*}
&1110000\\
&1001100\\
&1000011\\
&0101001\\
&0100110\\
&0011010\\
&0010101\\
\end{align*}
9
\begin{align*}
&1110000\\
&1001001\\
&1000110\\
&0101100\\
&0100011\\
&0011010\\
&0010101\\
\end{align*}
10
\begin{align*}
&1101000\\
&1010001